In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
import pandas as pd

/Users/meetkumarpatel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
classifier = Sequential()
classifier.add(Conv2D(filters=32, kernel_size=(3,3),strides=(1, 1), input_shape=(64,64,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(rate=0.2))
classifier.add(Dense(units=1, activation='sigmoid'))
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

In [3]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipvalue=0.5)
classifier.compile( optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
from keras.preprocessing.image import ImageDataGenerator
# applying transformation to image
train_datagen = ImageDataGenerator(
 rescale=1./255,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
training_set = train_datagen.flow_from_directory(
        '/users/meetkumarpatel/desktop/fingerprint_model/dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        '/users/meetkumarpatel/desktop/fingerprint_model/dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 14848 images belonging to 2 classes.
Found 4984 images belonging to 2 classes.


In [ ]:
from IPython.display import display 

history = classifier.fit_generator(
                training_set,
                steps_per_epoch=250,
                epochs=15,
                validation_data=test_set, validation_steps=150)

In [10]:
classifier.save("fingerprint_1.h5",overwrite=True)
#classifier.summary()


In [14]:
training_set.class_indices

{'no': 0, 'yes': 1}

# test

In [17]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img("/users/meetkumarpatel/desktop/fingerprint_test_yes.jpg",target_size=(64, 64) )
# Adding the channel
test_image = image.img_to_array(test_image)
# adding the batch size as predict method expects
test_image = np.expand_dims(test_image, axis=0)
# Predicting the test image
result= classifier.predict(test_image)
print(result)

[[1.]]


In [18]:
print(result[0][0])

1.0
